In [133]:
import pandas as pd 
import numpy as np

visa_experience = '/Users/churnika/Desktop/Projects/ML-Crate/F1 Visa Experiences/Dataset/telegram.csv'
visa_data = pd.read_csv(visa_experience) 

visa_data_clean = visa_data.dropna() #removing rows with missing values

visa_data_clean.describe(include='all')

,msg_id,date,message
count,6391.000000,6391,6391
unique,NaN,6372,6317
top,NaN,2021-07-01 04:30:10,Look this out for visa alerts👇\nhttps://t.me/A...
freq,NaN,2,5
mean,3384.868252,NaN,NaN
std,1908.469258,NaN,NaN
min,4.000000,NaN,NaN
25%,1768.500000,NaN,NaN
50%,3401.000000,NaN,NaN
75%,5034.500000,NaN,NaN


In [134]:
# accessing the message column and converting it into lower case

msg_exp = visa_data_clean['message'].str.lower()

In [135]:
print(msg_exp)

0       consulate: new delhi\nfirst attempt\nbiometric...
1       date: 24-03-2022\ndelhi consulate\ninterview t...
2       first attempt rejected in mumbai \ndelhi consu...
3       date 17 march 2022\nlocation- new delhi\ncount...
4       location mumbai\ndate 21 march\ncounter 33\nht...
                              ...                        
6438    consulate- mumbai\ndate-27th october\nappointm...
6439    consulate: hyderabad\ncounters open: 1\ncounte...
6440    location: mumbai\ndate : 23 oct 2020\napproved...
6441    kolkata counsulate \n8.0 am slot\ndate:19th oc...
6442    visa #experience\nuniversity: stonybrook unive...
Name: message, Length: 6391, dtype: object


In [136]:
# removing the urls that are present.

import re

text_without_urls = []

def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    text_without_urls = url_pattern.sub('',text)
    return text_without_urls

for i in range(len(msg_exp)):
    try:
        msg_exp[i] = remove_urls(msg_exp[i])
        text_without_urls.append(msg_exp[i])
    except KeyError as e:
        # print(f"Error at index {i}: {e}. Skipping...")
        continue


In [137]:
# removing punctuations

import string

def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

print("Length of text_without_urls:", len(text_without_urls))

for i in range(len(text_without_urls)):
    try:
        text_without_urls[i] = remove_punctuation(text_without_urls[i])

    except KeyError as e:
        print(f"Error at index {i}: {e}. Skipping...")
        continue

Length of text_without_urls: 6340


In [188]:
# TOKENIZATION
# REMOVING URLS
# REMOVING NUMBERS
# REMOVING STOP WORDS

import nltk
import re
from nltk.corpus import stopwords
import calendar

nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

months_days = [calendar.month_name[i].lower() for i in range(1, 13)] + [calendar.day_name[i].lower() for i in range(7)]
remove_words = ["vo","n","m","c","ra","xx","r","date","hii","hi","ye","pa","xxx","p","sir","mam","good","morning","time","ur","you","status","father"]

cleaned_tokens = []

for i in range(len(text_without_urls)):
    try:
        tokens = nltk.word_tokenize(text_without_urls[i])
        # Filter out URLs
        tokens = [token for token in tokens if not re.match(r'http[s]?://', token)]
        tokens = [token for token in tokens if token.isalpha()]
        tokens = [token for token in tokens if token not in stop_words]
        tokens = [token for token in tokens if token not in months_days]
        tokens = [token for token in tokens if token not in remove_words]
        cleaned_tokens.append(tokens)
    except Exception as e:
        print(f"Error at index {i}: {e}. Skipping...")
        continue


[nltk_data] Downloading package punkt to /Users/churnika/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/churnika/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [181]:
# LEMMATIZATION

from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

lemmatized_tokens = [[lemmatizer.lemmatize(token) for token in tokens]for tokens in cleaned_tokens]

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/churnika/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [182]:
# REMOVING RARE OR FREQUENTLY USED WORDS

from collections import Counter

freq = Counter([token for tokens in lemmatized_tokens for token in tokens])
freq_threshold = 10

lemmatized_tokens = [[token for token in tokens if freq[token] > freq_threshold] for tokens in lemmatized_tokens]

In [183]:
#REMOVING WHITESPACES

cleaned_lemmatized_tokens = [' '.join(tokens) for tokens in lemmatized_tokens]
cleaned_lemmatized_tokens = [text.split() for text in cleaned_lemmatized_tokens]

In [184]:
# Dictionary mapping abbreviations to their full forms
abbreviation_dict = {
    "don't": "do not",
    "can't": "cannot",
    "u": "you",
    "approv": "approve",
    "approved": "approve",
    "rejected": "reject"
    # Add more abbreviations and their full forms as needed
}

# Function to expand abbreviations
def expand_abbreviations(tokens):
    expanded_tokens = []
    for token in tokens:
        if token in abbreviation_dict:
            expanded_tokens.extend(abbreviation_dict[token].split())
        else:
            expanded_tokens.append(token)
    return expanded_tokens

# Expand abbreviations in cleaned_lemmatized_tokens
expanded_lemmatized_tokens = [expand_abbreviations(tokens) for tokens in cleaned_lemmatized_tokens]


In [185]:
#REMOVE ACCENTS AND DIACRITICS FROM EXPANDED_LEMMATIZED_TOKENS

from unidecode import unidecode

def remove_accents_diacritics(tokens):
    cleaned_tokens = []
    for token in tokens:
        cleaned_token = unidecode(token)
        cleaned_tokens.append(cleaned_token)
    return cleaned_tokens

expanded_lemmatized_tokens = [remove_accents_diacritics(tokens) for tokens in expanded_lemmatized_tokens]


In [189]:
# PART-OF-SPEECH

pos_tagged_tokens = [nltk.pos_tag(tokens) for tokens in expanded_lemmatized_tokens]

In [194]:
# VECTORIZATION

from sklearn.feature_extraction.text import TfidfVectorizer

joined_tokens = [' '.join(tokens) for tokens in expanded_lemmatized_tokens]

vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform(joined_tokens)

In [209]:
# SENTIMENT ANALYSIS

from nltk.sentiment import SentimentIntensityAnalyzer

# Create SentimentIntensityAnalyzer object
sia = SentimentIntensityAnalyzer()

# Let's say we have the following tokens
tokens = expanded_lemmatized_tokens

# Function to get sentiment
def get_sentiment(token_list):
    # Join the tokens into a string
    text = ' '.join(token_list)
    # Use the polarity_scores method to get the sentiment
    sentiment = sia.polarity_scores(text)
    return sentiment

# Apply the function to each list of tokens
sentiments = [get_sentiment(token_list) for token_list in tokens]

# Now, 'sentiments' is a list of sentiment scores for each message


In [210]:
# MAPPING SENTIMENTS TO y

def map_sentiment_score_to_label(sentiment):
    if sentiment['compound'] >= 0.05:
        return 'positive'
    elif sentiment['compound'] <= -0.05:
        return 'negative'
    else:
        return 'neutral'

# Map sentiment scores to sentiment labels
sentiment_labels = [map_sentiment_score_to_label(sentiment) for sentiment in sentiments]

y = sentiment_labels


In [211]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)